In [1]:
# imports
import os

import pandas as pd
import numpy as np
from sklearn.metrics import classification_report, f1_score, recall_score

In [2]:
# configs
PROJECT_PATH = "/home/alex/paper-2025-anonymous-submission"

In [3]:
# load data

dataset = pd.read_json(
    os.path.join(
        PROJECT_PATH,
        "Data/processed_data/dataset.json"
    ),
    orient="index"
)

dev_dataset = pd.read_json(
    os.path.join(
        PROJECT_PATH,
        "Data/processed_data/dev_dataset.json"
    ),
    orient="index"
)

/home/alex/aspirantura_hse/general_venv/lib/python3.10/site-packages/pandas/core/tools/datetimes.py:557: RuntimeWarning: invalid value encountered in cast
  arr, tz_parsed = tslib.array_with_unit_to_datetime(arg, unit, errors=errors)
/home/alex/aspirantura_hse/general_venv/lib/python3.10/site-packages/pandas/core/tools/datetimes.py:557: RuntimeWarning: invalid value encountered in cast
  arr, tz_parsed = tslib.array_with_unit_to_datetime(arg, unit, errors=errors)


In [4]:
df = pd.concat([dataset, dev_dataset])

In [5]:
df

,annotations,summary,is_word_play,date,article_url,headline,lead
0,None,Жилищный нацпроект помимо количественных целей...,False,2021-12-16,https://www.kommersant.ru/doc/5130217,Комфортная среда,Городскую среду улучшают с учетом мнения жителей
1,"[{'headline_substring': 'стушевалась', 'start_...",Окончание летних каникул и сезона шашлыков при...,True,2021-08-09,https://www.kommersant.ru/doc/4976692,Свинина стушевалась,Началось сезонное снижение цен
2,"[{'headline_substring': '«Истока»', 'start_ind...","Как стало известно “Ъ”, главное военное следст...",True,2023-09-11,https://www.kommersant.ru/doc/6210688,Исток с «Истока»,Раскрыта преступная группа несунов с оборонног...
3,"[{'headline_substring': 'Налетай, позеленело',...","Как выяснил “Ъ”, крупнейший инвестор в солнечн...",True,2021-04-08,https://www.kommersant.ru/doc/4928249,"Налетай, позеленело",Компания «Хевел» выставила электроэнергию со с...
4,[{'headline_substring': 'Гриппы по осени счита...,В России стартовала кампания прививок против г...,True,2023-09-11,https://www.kommersant.ru/doc/6210686,Гриппы по осени считают,В России началась массовая вакцинация от вируса
...,...,...,...,...,...,...,...
195,"[{'headline_substring': 'подошли', 'start_inde...",Основными покупателями стартапов в 2021 году с...,True,2021-08-12,https://www.kommersant.ru/doc/5117889,К стартапам подошли экосистемно,Корпорации начали активнее инвестировать в нов...
196,[{'headline_substring': 'Звезда по имени Моди'...,Новость об успешной посадке на Луне индийской ...,True,2023-08-24,https://www.kommersant.ru/doc/6174473,Звезда по имени Моди,Обозреватель “Ъ” Сергей Строкань — о новом гло...
197,None,"В Кемерово суд арестовал Михаила Федяева, през...",False,2021-12-15,https://www.kommersant.ru/doc/5130733,Собственник подорвался на шахте,По делу о ЧП на «Листвяжной» арестован президе...
198,None,Авторынок РФ в августе продолжил медленное вос...,False,2022-08-30,https://www.kommersant.ru/doc/5535961,Авторынок растет на льготных кредитах,Lada обеспечила более трети продаж в августе


In [6]:
def extract_annotations_labels(annotations):
    
    all_wordplay_type = list()
    
    if isinstance(annotations, list):
        for annot in annotations:
            if isinstance(annot, dict):
                all_wordplay_type.append(
                    annot["wordplay_type"]
                )
            elif isinstance(annot, list):
                all_wordplay_type.append(
                    annot[0]["wordplay_type"]
                )
    
    return all_wordplay_type


In [7]:
wikipedia_urls = 0
witionary_urls = 0

In [8]:
def extract_annotations_labels_with_links(annotations):

    global wikipedia_urls
    global witionary_urls
    
    all_wordplay_type = list()
    
    if isinstance(annotations, list):
        for annot in annotations:
            if isinstance(annot, dict) and "http" in annot["reference_url"]:
                all_wordplay_type.append(
                    annot["wordplay_type"]
                )
                if annot["wordplay_type"] in ["Collocation", "Reference", "Idiom"]:
                    if "wikipedia" in annot["reference_url"]:
                        wikipedia_urls += 1
                    else:
                        witionary_urls += 1
            elif isinstance(annot, list) and "http" in annot[0]["reference_url"]:
                all_wordplay_type.append(
                    annot[0]["wordplay_type"]
                )

                if annot[0]["wordplay_type"] in ["Collocation", "Reference", "Idiom"]:
                    if "wikipedia" in annot[0]["reference_url"]:
                        wikipedia_urls += 1
                    else:
                        witionary_urls += 1
    
    return all_wordplay_type


In [9]:
df["all_wordplay_type"] = df["annotations"].apply(extract_annotations_labels)


In [12]:
wikipedia_urls, witionary_urls

(290, 168)

In [11]:
df["all_wordplay_type_link"] = df["annotations"].apply(extract_annotations_labels_with_links)

In [13]:
df["all_wordplay_type"].explode().value_counts()

Collocation            424
Reference              352
Polysemy               190
Nonce word             185
Idiom                  177
Phonetic similarity     98
Oxymoron                48
Homonymy                26
Name: all_wordplay_type, dtype: int64

In [14]:
df["all_wordplay_type_link"].explode().value_counts()

Reference              213
Collocation            127
Idiom                  118
Homonymy                 4
Polysemy                 2
Nonce word               2
Phonetic similarity      2
Name: all_wordplay_type_link, dtype: int64

In [15]:
def extract_links(annotations):
    
    all_wordplay_type = list()
    
    if isinstance(annotations, list):
        for annot in annotations:
            if isinstance(annot, dict) and "http" in annot["reference_url"]:
                all_wordplay_type.append(
                    annot["reference_url"]
                )
            elif isinstance(annot, list) and "http" in annot[0]["reference_url"]:
                all_wordplay_type.append(
                    annot[0]["reference_url"]
                )
    
    return all_wordplay_type


In [16]:
df["all_links"] = df["annotations"].apply(extract_links)

In [17]:
len(df["all_links"].explode().dropna().tolist())

468

In [62]:
df[["headline", "all_wordplay_type"]].sample(3)

,headline,all_wordplay_type
2343,Иноагент остался без прецедента,[]
64,Батареи просят рубля,[]
260,Штамм «дельта» пробил Великую иммунную стену,[Collocation]


In [63]:
df["all_wordplay_type_len"] = df["all_wordplay_type"].apply(lambda x: len(x))

In [64]:
df["all_wordplay_type_len_set"] = df["all_wordplay_type"].apply(lambda x: len(set(x)))

In [66]:
len(df[df["all_wordplay_type_len"] != df["all_wordplay_type_len_set"]])

29